# To locate FIT and LOT in large documents

In [24]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import json
import openai
import os

with open('../local_settings.json') as f:
    data = json.load(f)

# Set form recogniser client
credential = AzureKeyCredential(data["FORM_KEY"])
document_analysis_client = DocumentAnalysisClient(data["FORM_ENDPOINT"], credential)

# This example also requires an OpenAI API key
os.environ['OPENAI_API_KEY'] = data['OPENAI_API_KEY']
openai.api_key = os.environ['OPENAI_API_KEY']

In [25]:
field_names = ["Mission", "Colour Palette"]
field_descriptions = ["", ""]

json_template = json.dumps({"Results": [dict(zip(field_names, field_descriptions))]})


system = f"""
You are an assistant that given a text extracted using OCR from a document will extract user provided data fields.
Fields can have multiple formats.
Write your output as a JSON with an entry with the format {json_template} per each test you find.
If there is a field that you can not find, set it a null.
If there is any additional information of feedback from the infromation extraction, add a {{"notes": "<additional-information>"}}
"""   # noqa E501

In [26]:
import pdfplumber

path = "../data/example/Elastacloud-Brand-Book.pdf"

relevant_text = ""
with pdfplumber.open(path) as pdf:
    for page in pdf.pages:
        relevant_text += pdf.pages[page.page_number-1].extract_text()

In [27]:
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.node_parser import SimpleNodeParser
from llama_index import Document
from langchain.chat_models import ChatOpenAI
from llama_index import LLMPredictor


documents = [Document(text=relevant_text)]

node_parser = SimpleNodeParser.from_defaults(chunk_size=4096,
                                                chunk_overlap=200)
                                                
llm = ChatOpenAI(temperature=0, max_tokens=512)
llm_predictor = LLMPredictor(llm=llm)
service_context = ServiceContext.from_defaults(node_parser=node_parser,
                                                llm_predictor=llm_predictor)

index = VectorStoreIndex.from_documents(documents,
                                        service_context=service_context)
query_engine = index.as_query_engine()

In [28]:
response = query_engine.query("What is EC color palette?")

In [33]:
response.metadata

{'d696ffca-125f-4ab1-bad9-f1d7f82c0572': {},
 'e2aeb521-8bc5-496c-9624-0a928c435b5e': {}}

In [35]:
response.source_nodes.

<function list.index(value, start=0, stop=9223372036854775807, /)>

## PDF retrieval tool

In [9]:
# read a pdf and answers question on it
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/example/Elastacloud-Brand-Book.pdf")
pages = loader.load_and_split()
# check the metadata
pg_no=6
print(f"Content:\n {pages[pg_no].page_content} \n metadata:\n {pages[pg_no].metadata}")

Content:
 A culture of giving back
Elastacloud firmly believes that data can break boundaries 
and benefit humanity, which is why we are heavily 
invested in projects and actions that improve outcomes 
for individuals and the planet. One way we do this is by 
empowering our Elastaclouders to contribute to the wider 
community through sharing their digital skills.For well over a decade, Elastacloud has established and 
maintained various industry user groups: joining together 
more than 12,000 people around the world for community-
based learning and innovation. This includes one of the 
largest data science communities in Europe.
We are also proud to sponsor computer labs at Shanti 
Bhavan, a residential school in India for children born into 
the lowest socioeconomic class, which aims to uplift from 
poverty through education and opportunity. Our team in 
India run coding labs for the students there, sometimes 
held in the Elastacloud offices. 
Elastacloud has a deeply embedded commit

In [10]:
loader

In [19]:
pages

[Document(page_content='BRAND BOOK V4\nMarch 2023', metadata={'source': '../data/example/Elastacloud-Brand-Book.pdf', 'page': 0}),
 Document(page_content='WHO WE ARE\nOUR VISION\nGlobal industry transformation through data and next generation AI\nWe believe every business has immense power in its data. The power to unlock growth, to unleash intelligence, and to accelerate \noutcomes – responsibly, and with the help of AI.\nWe’re playing in a new game –where data is the fuel for your future growth, and AI is the accelerator to your destination.\nIn this new era of AI, the power of data and AI is no longer limited to the few companies with skilled data scientists and experts. \nNow, every business has opportunities to do things better, smarter, faster – to become more intelligent.\nElastacloud will lead this new era and help democratise intelligence – empowering businesses to grow sustainably –  \nby unleashing the power of their data with intelligent solutions and services.\nOUR MISSION

In [21]:
type(pages[0])

langchain.schema.document.Document

In [ ]:
from langchain.document import Document

doc = Document(page_content="foo")

doc_with_metadata = Document(page_content="foo", metadata={"source": "1"})


In [18]:
type(pages)

list

In [11]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.chains import create_qa_with_sources_chain
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma


# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(pages, embedding_function)

c:\Users\AntonioVelasco\.virtualenvs\llm_utilities-HQ1bCkrd\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.chains import create_qa_with_sources_chain, RetrievalQA
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma


# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(pages, embedding_function)


llm_src = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

qa_chain = create_qa_with_sources_chain(llm_src)

doc_prompt = PromptTemplate(
    template="Content: {page_content}\n Source: {page}", # look at the prompt does have page#
    input_variables=["page_content", "source"],
)

final_qa_chain = StuffDocumentsChain(
    llm_chain=qa_chain, 
    document_variable_name='context',
    document_prompt=doc_prompt,
)
retrieval_qa = RetrievalQA(
    retriever=db.as_retriever(),
    combine_documents_chain=final_qa_chain
)

In [34]:

query = "What is EC Mission?"

answer_1 = retrieval_qa.run(query)
print(answer_1)

{
  "answer": "Elastacloud's mission is to help businesses achieve their goals faster by delivering actionable insights and more efficient processes and services.",
  "sources": ["21"]
}


In [31]:
json.loads(answer_1)["answer"]

"Elastacloud's mission is to be data pioneers, unleashing the power of data, accelerating transformational outcomes, serving as trusted advisors, and fostering a global community of innovation and collaboration."

In [22]:
import json
field_names = ["mission", "Colour"]
field_descriptions = ["<>", "<Elastacloud marketing palette>"]

json_template = json.dumps({"Results": dict(zip(field_names, field_descriptions))})
json_template

'{"Results": {"mission": "<>", "Colour": "<Elastacloud marketing palette>"}}'

In [23]:
json_template = json.dumps(
    {
        "Results": [
            dict(list(zip(field_names, field_descriptions)) + [("Source page", "<Page number>")])
        ]
    }
)

print(json_template)

{"Results": [{"mission": "<>", "Colour": "<Elastacloud marketing palette>", "Source page": "<Pages of the document containing the fields in int format>"}]}


In [28]:
b=""
system = f"""
You are an assistant tasked with extracting data from documents.
Given a text extracted using OCR from a document, you will extract a user provided data field.
Be thorough and extract all the information available.
Write your output as a JSON with the format {b}.
If you can't find the asked information, set it null.
If there is any additional information of feedback from the infromation extraction, add a {{"notes": "<additional-information>"}}
"""   # noqa E501

for a in json_template:
    b = json.dumps({"Results": [dict(zip([a], ["<Extracted answer>"]))]})
    print(b)

{"Results": [{"mission": "<Extracted answer>"}]}
{"Results": [{"Colour": "<Extracted answer>"}]}


## LLM call

In [ ]:
input = relevant_text

json_template = json.dumps({ 
    "Test Type": "<FIT or LOT>",
    "Casing Shoe": "<Casing shoe size>",
    "TVD (m)": "TVD in meters",
    "Surface pressure (psi)": "<MW value in sg>",
    "MW (sg)": "<MW value in sg>",
    "EMW (sg)": "<EMW value in sg. Calculated as: EMW = MW + (P-FIT / (TVD-shoe * 1.421))>"
})

system = f"""
You are an API that given a text extracted using OCR from an End of Well Report will extract Formation Integrity Test (FIT) and Leak Off Test (LOT) results.
Your response will be a JSON with as many entries as needed in the format {json_template}
If there is a field that you can not find, set it a null.
If the document has any kind of errors or is corrupted, add a field {{"errors": "<error description>"}}
If there is any additional information of feedback from the infromation extraction, add a {{"notes": "<additional-information>"}}

The format of your input will be the text of the relevant page.
"""

config = {
    "temperature": 0.2,
    "max_tokens": 512,
    "top_p": 1,
}

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": system},
      {"role": "user", "content": input},
    ],
    temperature=config["temperature"],
    max_tokens=config["max_tokens"],
    top_p=config["top_p"],
  )

print(response.get("choices")[0]["message"]["content"])

In [241]:
input = relevant_text

json_template = json.dumps({ 
    "Test Type": "<FIT or LOT>",
    "Casing Shoe": "<Casing shoe size>",
    "TVD (m)": "TVD in meters",
    "Surface pressure (psi)": "<MW value in sg>",
    "MW (sg)": "<MW value in sg>",
    "EMW (sg)": "<EMW value in sg. Calculated as: EMW = MW + (P-FIT / (TVD-shoe * 1.421))>"
})

system = f"""
You are an API that given a text extracted using OCR from an End of Well Report will extract Formation Integrity Test (FIT) and Leak Off Test (LOT) results.
Your response will be a JSON with as many entries as needed in the format {json_template}
If there is a field that you can not find, set it a null.
If the document has any kind of errors or is corrupted, add a field {{"errors": "<error description>"}}
If there is any additional information of feedback from the infromation extraction, add a {{"notes": "<additional-information>"}}

The format of your input will be the text of the relevant page.
"""

config = {
    "temperature": 0.2,
    "max_tokens": 512,
    "top_p": 1,
}

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages=[
      {"role": "system", "content": system},
      {"role": "user", "content": input},
    ],
    temperature=config["temperature"],
    max_tokens=config["max_tokens"],
    top_p=config["top_p"],
  )

print(response.get("choices")[0]["message"]["content"])

{"text": "Formation Integrity Test (FIT)\n\nCasing Shoe: 10000 ft\nTVD (m): 3000\nSurface pressure (psi): 5000\nMW (sg): 1.2\n\nLeak Off Test (LOT)\n\nCasing Shoe: 10000 ft\nTVD (m): 3000\nSurface pressure (psi): 5000\nMW (sg): 1.2\nEMW (sg): 1.3"}


Good enough with 16k, let's check with form recogniser and see 

In [ ]:
len(encoder.encode(extracted_text["content"]))

In [ ]:
input = extracted_text["content"]

json_template = json.dumps({ 
    "Test Type": "<FIT or LOT>",
    "Casing Shoe": "<Casing shoe size>",
    "TVD (m)": "TVD in meters",
    "Surface pressure (psi)": "<MW value in sg>",
    "EMW (sg)": "<EMW value in sg>"
})

system = f"""
You are an API that given a text extracted using OCR from an End of Well Report will extract Formation Integrity Test (FIT) and Leak Off Test (LOT) results.
There can be multiple tests per shoe size and depth, please report them all.
Your response will be a JSON with one entry per test in the format {json_template}.
If there is a field that you can not find, set it a null.
If the document has any kind of errors or is corrupted, add a field {{"errors": "<error description>"}}
If there is any additional information of feedback from the infromation extraction, add a {{"notes": "<additional-information>"}}

The format of your input will be the text of the relevant page.
"""

config = {
    "temperature": 0.2,
    "max_tokens": 512,
    "top_p": 1,
}

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages=[
      {"role": "system", "content": system},
      {"role": "user", "content": input},
    ],
    temperature=config["temperature"],
    max_tokens=config["max_tokens"],
    top_p=config["top_p"],
  )

print(response.get("choices")[0]["message"]["content"])

# Implementing Llama Index 

In [244]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

In [245]:
documents = SimpleDirectoryReader("../llama_dir/").load_data()

In [246]:
index = VectorStoreIndex.from_documents(documents)

In [247]:
query_engine = index.as_query_engine()
response = query_engine.query("what's the EMW of the FIT performed on the well 206/12a-3?")
print(response)

The EMW of the FIT performed on the well 206/12a-3 is 1.75sg.


In [248]:
query_engine = index.as_query_engine()
response = query_engine.query("How many test where performend on the well 206/12a-3?")
print(response)

Two tests were performed on the well 206/12a-3.


In [249]:
query_engine = index.as_query_engine()
response = query_engine.query("In which page on the end of well 206/12a-3 report are the LOT and FIT reported?")
print(response)

The LOT and FIT are reported on page 53 of the end of well 206/12a-3 report.


# LLama Index Approach

In [ ]:
documents = SimpleDirectoryReader(
    input_files=[path]
).load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("In which page are the LOT and FIT reported?")
print(response)

In [242]:
json_template = json.dumps({
    "Test Type": "<FIT or LOT>",
    "Casing Shoe": "<Casing shoe size>",
    "TVD (m)": "TVD in meters",
    "Surface pressure (psi)": "<Surface pressure value>",
    "MW (sg)": "<MW value in sg>",
    "EMW (sg)": "<EMW value in sg>"
})

system = f"""
You are an assistant that given a text extracted using OCR from an End of Well Report will extract 'Formation Integrity Test' (FIT) and 'Leak Off Test' (LOT) results.
There can be multiple tests, report all of them.
Write your output as a list with an entry with the format {json_template} per each test you find, separated by commas.
If there is a field that you can not find, set it a null.
If the document has any kind of errors or is corrupted, add a field {{"errors": "<error description>"}}
If there is any additional information of feedback from the infromation extraction, add a {{"notes": "<additional-information>"}}
"""

In [243]:
response = query_engine.query(system)
print(response)

{"Test Type": "FIT", "Casing Shoe": "20", "TVD (m)": "536", "Surface pressure (psi)": "101.0", "MW (sg)": "1.15", "EMW (sg)": "1.26"},
{"Test Type": "LOT", "Casing Shoe": "20", "TVD (m)": "536", "Surface pressure (psi)": "84.0", "MW (sg)": "1.15", "EMW (sg)": "1.26"},
{"Test Type": "FIT", "Casing Shoe": "13 3/8", "TVD (m)": null, "Surface pressure (psi)": "328", "MW (sg)": "1.55", "EMW (sg)": null},
{"Test Type": "LOT", "Casing Shoe": "13 3/8", "TVD (m)": null, "Surface pressure (psi)": "360", "MW (sg)": "1.55", "EMW (sg)": null}


# Loading data from Form Recogniser

In [228]:
import io

# OCR from base form recogniser
def base_form_recogniser(pdf_bytes: io.BytesIO) -> dict:
    document = pdf_bytes.getvalue()

    # Start the document analysis
    poller = document_analysis_client.begin_analyze_document("prebuilt-document", document, polling_interval=5)

    # Get the result
    result = poller.result()
    data = result.to_dict()
    return data

In [229]:
from PyPDF4 import PdfFileWriter, PdfFileReader

inputpdf = PdfFileReader(open(path, "rb"))

output = PdfFileWriter()

relevant_text = ""

with pdfplumber.open(path) as pdf:
    for page in pdf.pages:
        if page.search("FIT") and page.search("LOT"):
            output.addPage(inputpdf.pages[page.page_number])
            print(page.page_number)

output_bytesio = io.BytesIO()

output.write(output_bytesio)

4
5
8
13
15
52
53
54


In [230]:
extracted_text = base_form_recogniser(output_bytesio)

In [231]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex, ServiceContext
from llama_index.node_parser import SimpleNodeParser
from llama_index import Document

text_list = [extracted_text["content"]]
documents = [Document(text=t) for t in text_list]

node_parser = SimpleNodeParser.from_defaults(chunk_size=4096, chunk_overlap=200)
service_context = ServiceContext.from_defaults(node_parser=node_parser)

index = VectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine()

In [239]:
json_template = json.dumps({
    "Test Type": "<FIT or LOT>",
    "Casing Shoe": "<Casing shoe size>",
    "TVD (m)": "TVD in meters",
    "Surface pressure (psi)": "<Surface pressure value>",
    "MW (sg)": "<MW value in sg>",
    "EMW (sg)": "<EMW value in sg>"
})

system = f"""
You are an assistant that given a text extracted using OCR from an End of Well Report will extract 'Formation Integrity Test' (FIT) and 'Leak Off Test' (LOT) results.
You must prioritize reporting the EMW value, meassured in sg.
There can be multiple tests, report all of them.
Write an entry with the format {json_template} per each test you find.
If there is a field that you can not find, set it a null.
If the document has any kind of errors or is corrupted, add a field {{"errors": "<error description>"}}
If there is any additional information of feedback from the infromation extraction, add a {{"notes": "<additional-information>"}}
"""

In [240]:
response = query_engine.query(system)
print(response)

{"Test Type": "FIT", "Casing Shoe": "9 5/8", "TVD (m)": "2081.4", "Surface pressure (psi)": "null", "MW (sg)": "null", "EMW (sg)": "1.69"}
{"Test Type": "LOT", "Casing Shoe": "null", "TVD (m)": "536", "Surface pressure (psi)": "84.0", "MW (sg)": "null", "EMW (sg)": "1.26"}
{"Test Type": "LOT", "Casing Shoe": "null", "TVD (m)": "null", "Surface pressure (psi)": "null", "MW (sg)": "null", "EMW (sg)": "null"}
{"Test Type": "FIT", "Casing Shoe": "13 3/8", "TVD (m)": "null", "Surface pressure (psi)": "null", "MW (sg)": "null", "EMW (sg)": "1.55"}
{"Test Type": "FIT", "Casing Shoe": "null", "TVD (m)": "null", "Surface pressure (psi)": "null", "MW (sg)": "null", "EMW (sg)": "null"}
